# Segmenting and Clustering Neighborhoods in London

### Part:1 creating a new Notebook for this assignment and dataframe should consist of three columns: Postcode, Borough, and Neighborhood

### Import Required libraries

In [1]:
import requests 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans # import k-means from clustering stage
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors
import folium 
import toolforge
print ("Libraries Imported")

Libraries Imported


Your credentails:
CLIENT_ID: 3WAFTWT02PF3OQSVM0QQYVCJQXRHQL3GCEFUEI5TEJQEA4TC
CLIENT_SECRET:VZEMGEOKJDOWS4WT1LSMD3Z5OCMYXC3OTEAJ1JRUBGJEJYBP


### Access the url & print the required table using request.get function

In [3]:
source=requests.get("https://en.wikipedia.org/wiki/List_of_London_boroughs").text
soup=BeautifulSoup(source,"lxml") 
#print (soup)

### Create a for loop for itterating through the table to find the table data(Borough and Co-ordinates)

In [4]:
df_list = []

for items in soup.find('table', class_= 'wikitable sortable').find_all('tr')[1::]: # "tr" is tag of html for defining table rows
    data = items.find_all(['td']) # td tag in html used to define the data in table
    data1 = data[0] # 1st column i.e. Borough
    data8 = data[8] # 9th column i.e Co-ordinates
    try:
        borough_name = data1.get_text()
        borough_name = borough_name.split('[') # it will break the given string by the specified separator(where '[' is mentioned in the string)        
        borough_name = borough_name[0]
        borough_name = borough_name.strip() #need to remove all the leading and trailing spaces from a string.
        
        # Co-ordinate data on portal is in DMS,decimal,geo uri,.... formats respectively, We need only decimal format 
        #51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /
        
        ll = data8.get_text()
        ll = ll.split('/') #it will break the given string by the specified separator.
        lat_long = ll[2]  # output format 51.5607; 0.1557﻿ (Barking and Dagenham)\n
        lat_long = lat_long.split('(') # output format 51.5607; 0.1557﻿ , Barking and Dagenham)\n
        lat_long = lat_long[0] # output format 51.5607; 0.1557﻿
        lat_long = lat_long.split(';') # output format [ 51.5607, 0.1557﻿ ]
        latitude = lat_long[0] # output format 51.5607
        latitude = latitude.strip() # output format 51.5607
        longitude = lat_long[1] # output format 0.1557﻿
        longitude = longitude.strip() # output format 0.1557﻿
        longitude = longitude.replace(u'\ufeff', '')
        latitude = float(latitude) #convert latitude to float
        longitude = float(longitude) # convert longitude to type float

#       Append the borough name, latitude and logitude in a list
        df_list.append((borough_name,latitude, longitude))
    except IndexError:pass

In [5]:
df_data = pd.DataFrame(df_list, columns=['Borough', 'Latitude' , 'Longitude'])
print (df_data.shape)
print (df_data.dtypes)
df_data.head()

(32, 3)
Borough       object
Latitude     float64
Longitude    float64
dtype: object


,Borough,Latitude,Longitude
0,Barking and Dagenham,51.5607,0.1557
1,Barnet,51.6252,-0.1517
2,Bexley,51.4549,0.1505
3,Brent,51.5588,-0.2817
4,Bromley,51.4039,0.0198


### Get the Latitude and Longitude of London City using geopy library

In [6]:
address = 'London'

geolocator = Nominatim(user_agent="London")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


### Create a map of London with Boroughs superimposed on top.

In [7]:
map_London= folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough in zip(df_data['Latitude'], df_data['Longitude'], df_data['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_London)  
    
map_London

In [8]:
df_data.loc[df_data['Borough'] == 'Newham']

,Borough,Latitude,Longitude
23,Newham,51.5077,0.0469


## Preferred location for Asian restuarant - Newham Borough

As the borough Newham is having high asian population, we will consider to Segment and Cluster only the neighbourhoods of this borough. For that we have to get the latitude and longitude details of all the areas(neighbourhoods) of Newham borough.

Read the latitude and longitude coordinates of all the neighborhoods(areas) in Newham Borough

In [9]:
from urllib.request import urlopen
import re

source2 = requests.get(r"https://en.wikipedia.org/wiki/List_of_areas_of_London").text
soup = BeautifulSoup(source2,'lxml')

codes = []
areas_list = []

for items in soup.find('table', class_= 'wikitable sortable').find_all('tr')[1::]:
    data = items.find_all(['td'])
    data0 = data[0]
    area_name = data0.text

    data1 = data[1]
    data1 = data1.text
    borough = data1.split('[')
    borough_name = borough[0]
    data5 = data[5]
    code = data5.text
    code = code.strip()
    
    if borough_name == 'Newham':
        codes.append(code)
        areas_list.append((borough_name,area_name))

In [10]:
df_areas = pd.DataFrame(areas_list, columns=['Borough', 'Area'])
print (df_areas.shape)
df_areas.head()

(14, 2)


,Borough,Area
0,Newham,Beckton
1,Newham,Canning Town
2,Newham,Custom House
3,Newham,East Ham
4,Newham,Forest Gate


In [11]:
from opencage.geocoder import OpenCageGeocode
key = "68a7a7ad0e964bd895e494c3adde8855"  # get api key from:  https://opencagedata.com
geocoder = OpenCageGeocode(key)
query = 'Newham London'  
results = geocoder.geocode(query)
#print (results)
lat = results[0]['geometry']['lat']
lng = results[0]['geometry']['lng']
print (lat, lng)

51.5299995 0.029318


In [12]:
list_lat = []   # create empty lists
list_lng = []

for index, row in df_areas.iterrows(): # iterate over rows in dataframe
    Area = row['Borough']
    Code = row['Area']       
    query = str(Area)+','+str(Code)

    results = geocoder.geocode(query)   
    lat = results[0]['geometry']['lat']
    lng = results[0]['geometry']['lng']

    list_lat.append(lat)
    list_lng.append(lng)

# create new columns from lists    
df_areas['Latitude'] = list_lat   
df_areas['Longitude'] = list_lng

In [13]:
df_areas

,Borough,Area,Latitude,Longitude
0,Newham,Beckton,51.516080,0.059426
1,Newham,Canning Town,51.518174,0.013498
2,Newham,Custom House,34.646730,-103.904410
3,Newham,East Ham,51.532963,0.055320
4,Newham,Forest Gate,51.549524,0.024925
5,Newham,Little Ilford,51.550298,0.062522
6,Newham,Manor Park,51.550687,0.047639
7,Newham,Maryland,51.546053,0.005922
8,Newham,North Woolwich,51.500407,0.064154
9,Newham,Plaistow,51.529380,0.024910


In [14]:
Newham_borough = df_areas.rename(columns={'Area' :'Neighborhood'})
Newham_borough.head()

,Borough,Neighborhood,Latitude,Longitude
0,Newham,Beckton,51.516080,0.059426
1,Newham,Canning Town,51.518174,0.013498
2,Newham,Custom House,34.646730,-103.904410
3,Newham,East Ham,51.532963,0.055320
4,Newham,Forest Gate,51.549524,0.024925


In [15]:
Newham_borough.drop([2], axis=0, inplace=True)

In [16]:
Newham_borough.reset_index(drop=True)

,Borough,Neighborhood,Latitude,Longitude
0,Newham,Beckton,51.516080,0.059426
1,Newham,Canning Town,51.518174,0.013498
2,Newham,East Ham,51.532963,0.055320
3,Newham,Forest Gate,51.549524,0.024925
4,Newham,Little Ilford,51.550298,0.062522
5,Newham,Manor Park,51.550687,0.047639
6,Newham,Maryland,51.546053,0.005922
7,Newham,North Woolwich,51.500407,0.064154
8,Newham,Plaistow,51.529380,0.024910
9,Newham,Silvertown,51.501363,0.038518


In [17]:
Newham_borough.dtypes

Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [18]:
address = 'Newham, London'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Newham are {}, {}.'.format(latitude, longitude))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Newham are 51.52999955, 0.0293179602938221.


In [19]:
# create map of Newham using latitude and longitude values
map_Newham = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Newham_borough['Latitude'], Newham_borough['Longitude'], Newham_borough['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Newham)  
    
map_Newham

In [21]:
# check the first Neighborhood(Area) in Newham Borough
Newham_borough.loc[0]

Borough            Newham
Neighborhood      Beckton
Latitude          51.5161
Longitude       0.0594257
Name: 0, dtype: object

### Define Foursquare credentials

In [25]:
CLIENT_ID = '3WAFTWT02PF3OQSVM0QQYVCJQXRHQL3GCEFUEI5TEJQEA4TC' # your Foursquare ID
CLIENT_SECRET = 'VZEMGEOKJDOWS4WT1LSMD3Z5OCMYXC3OTEAJ1JRUBGJEJYBP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3WAFTWT02PF3OQSVM0QQYVCJQXRHQL3GCEFUEI5TEJQEA4TC
CLIENT_SECRET:VZEMGEOKJDOWS4WT1LSMD3Z5OCMYXC3OTEAJ1JRUBGJEJYBP


# Explore Neighborhoods in Newham Borough

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, 
            VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, lat, lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                             'Neighborhood Latitude', 
                             'Neighborhood Longitude', 
                             'Venue',
                             'Venue Latitude', 
                             'Venue Longitude',
                             'Venue Category']
    
    return(nearby_venues)

### Now write the code to run the above function on each neighborhood and create a new dataframe called Newham venues

In [42]:
Newham_venues = getNearbyVenues(names=Newham_borough['Neighborhood'],
                                   latitudes=Newham_borough['Latitude'],
                                   longitudes=Newham_borough['Longitude']
                                  )

print(Newham_venues.shape)
Newham_venues.head()

(191, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Beckton,51.51608,0.059426,East london Gymnastics Club,51.514107,0.060155,Gym / Fitness Center
1,Beckton,51.51608,0.059426,Home Bargains,51.516805,0.062804,Discount Store
2,Beckton,51.51608,0.059426,Lidl,51.515982,0.054794,Supermarket
3,Beckton,51.51608,0.059426,Lituanica,51.516442,0.062927,Grocery Store
4,Beckton,51.51608,0.059426,Premier Inn London Beckton,51.515017,0.060978,Hotel


In [44]:
#check how many venues were returned for each neighborhood
Newham_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Beckton,13,13,13,13,13,13
Canning Town,13,13,13,13,13,13
East Ham,15,15,15,15,15,15
Forest Gate,12,12,12,12,12,12
Little Ilford,1,1,1,1,1,1
Manor Park,4,4,4,4,4,4
Maryland,22,22,22,22,22,22
North Woolwich,6,6,6,6,6,6
Plaistow,6,6,6,6,6,6


In [45]:
# Let's find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(Newham_venues['Venue Category'].unique())))

There are 84 uniques categories.


## Analyze Each Neighborhood

In [46]:
# one hot encoding
Newham_onehot = pd.get_dummies(Newham_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Newham_onehot['Neighborhood'] = Newham_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Newham_onehot.columns[-1]] + list(Newham_onehot.columns[:-1])
Newham_onehot = Newham_onehot[fixed_columns]

Newham_onehot.head()

,Neighborhood,Accessories Store,Airport Terminal,Art Gallery,Asian Restaurant,Bakery,Bar,Boat or Ferry,Bookstore,Boutique,...,Sporting Goods Shop,Supermarket,Thai Restaurant,Theater,Toy / Game Store,Train Station,Tunnel,Turkish Restaurant,Warehouse Store,Waterfront
0,Beckton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beckton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Beckton,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Beckton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Beckton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [49]:
Newham_grouped = Newham_onehot.groupby('Neighborhood').mean().reset_index()
print(Newham_grouped.shape)
Newham_grouped.head()


(13, 85)


,Neighborhood,Accessories Store,Airport Terminal,Art Gallery,Asian Restaurant,Bakery,Bar,Boat or Ferry,Bookstore,Boutique,...,Sporting Goods Shop,Supermarket,Thai Restaurant,Theater,Toy / Game Store,Train Station,Tunnel,Turkish Restaurant,Warehouse Store,Waterfront
0,Beckton,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.153846,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
1,Canning Town,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.076923,0.000000,0.0
2,East Ham,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.066667,0.0,...,0.066667,0.000000,0.066667,0.0,0.0,0.000000,0.0,0.000000,0.066667,0.0
3,Forest Gate,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.083333,0.0,0.000000,0.000000,0.0
4,Little Ilford,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0


### Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [61]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Newham_grouped['Neighborhood']

for ind in np.arange(Newham_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Newham_grouped.iloc[ind, :], num_top_venues)


In [64]:
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(13, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beckton,Supermarket,Gym / Fitness Center,Clothing Store,Hotel,Grocery Store,Furniture / Home Store,Bus Station,Soccer Field,Shopping Plaza,Discount Store
1,Canning Town,Grocery Store,Fast Food Restaurant,Carpet Store,Breakfast Spot,Fish & Chips Shop,Movie Theater,Cosmetics Shop,Pizza Place,Gym,Turkish Restaurant
2,East Ham,Coffee Shop,Discount Store,Electronics Store,Pub,Park,Sandwich Place,Fast Food Restaurant,Optical Shop,Bookstore,Sporting Goods Shop
3,Forest Gate,Grocery Store,Fast Food Restaurant,Pub,Train Station,Bakery,Bus Stop,Fish & Chips Shop,Moving Target,Eastern European Restaurant,Dance Studio
4,Little Ilford,Park,English Restaurant,Cosmetics Shop,Dance Studio,Department Store,Dessert Shop,Discount Store,Doner Restaurant,Eastern European Restaurant,Electronics Store


# Cluster Neighborhoods

#### Run K-means algorithm to cluster the neighborhood into 5 clusters.

In [66]:
# set number of clusters
kclusters = 5
Newham_grouped_clustering = Newham_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Newham_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 1, 3, 0, 0, 2, 0])

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [67]:
Newham_merged = Newham_borough
# add clustering labels
Newham_merged['Cluster Labels'] = kmeans.labels_

# merge Neighborhoods dataframe with Newham borough dataframe to add latitude/longitude for each neighborhood
Newham_merged = Newham_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Newham_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Newham,Beckton,51.516080,0.059426,0,Supermarket,Gym / Fitness Center,Clothing Store,Hotel,Grocery Store,Furniture / Home Store,Bus Station,Soccer Field,Shopping Plaza,Discount Store
1,Newham,Canning Town,51.518174,0.013498,0,Grocery Store,Fast Food Restaurant,Carpet Store,Breakfast Spot,Fish & Chips Shop,Movie Theater,Cosmetics Shop,Pizza Place,Gym,Turkish Restaurant
3,Newham,East Ham,51.532963,0.055320,0,Coffee Shop,Discount Store,Electronics Store,Pub,Park,Sandwich Place,Fast Food Restaurant,Optical Shop,Bookstore,Sporting Goods Shop
4,Newham,Forest Gate,51.549524,0.024925,0,Grocery Store,Fast Food Restaurant,Pub,Train Station,Bakery,Bus Stop,Fish & Chips Shop,Moving Target,Eastern European Restaurant,Dance Studio
5,Newham,Little Ilford,51.550298,0.062522,1,Park,English Restaurant,Cosmetics Shop,Dance Studio,Department Store,Dessert Shop,Discount Store,Doner Restaurant,Eastern European Restaurant,Electronics Store


#### Let's Visualize the resulting clusters

In [71]:
Newham_grouped.shape

(13, 85)

In [68]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
y = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(y)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Newham_merged['Latitude'], Newham_merged['Longitude'], Newham_merged['Neighborhood'], Newham_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine the Clusters

In [72]:
Newham_merged.loc[Newham_merged['Cluster Labels'] == 0, Newham_merged.columns[[1] + list(range(5, Newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beckton,Supermarket,Gym / Fitness Center,Clothing Store,Hotel,Grocery Store,Furniture / Home Store,Bus Station,Soccer Field,Shopping Plaza,Discount Store
1,Canning Town,Grocery Store,Fast Food Restaurant,Carpet Store,Breakfast Spot,Fish & Chips Shop,Movie Theater,Cosmetics Shop,Pizza Place,Gym,Turkish Restaurant
3,East Ham,Coffee Shop,Discount Store,Electronics Store,Pub,Park,Sandwich Place,Fast Food Restaurant,Optical Shop,Bookstore,Sporting Goods Shop
4,Forest Gate,Grocery Store,Fast Food Restaurant,Pub,Train Station,Bakery,Bus Stop,Fish & Chips Shop,Moving Target,Eastern European Restaurant,Dance Studio
7,Maryland,Hotel,Pub,Grocery Store,Bus Stop,Café,Portuguese Restaurant,Liquor Store,Dance Studio,English Restaurant,Supermarket
8,North Woolwich,Waterfront,Pier,Boat or Ferry,Light Rail Station,Harbor / Marina,Tunnel,Toy / Game Store,Gym,Cosmetics Shop,Dance Studio
10,Silvertown,Airport Terminal,Tunnel,Park,Theater,Hotel,Italian Restaurant,Light Rail Station,Waterfront,English Restaurant,Department Store
11,Stratford,Pub,Hotel,Sandwich Place,Clothing Store,Shopping Mall,Italian Restaurant,Café,Pizza Place,Bookstore,Burger Joint
12,Upton Park,Grocery Store,Indian Restaurant,Bakery,Sandwich Place,Pub,Metro Station,Park,Bus Stop,Market,Shopping Mall


In [73]:
Newham_merged.loc[Newham_merged['Cluster Labels'] == 1, Newham_merged.columns[[1] + list(range(5, Newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Little Ilford,Park,English Restaurant,Cosmetics Shop,Dance Studio,Department Store,Dessert Shop,Discount Store,Doner Restaurant,Eastern European Restaurant,Electronics Store


In [74]:
Newham_merged.loc[Newham_merged['Cluster Labels'] == 2, Newham_merged.columns[[1] + list(range(5, Newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Plaistow,Bus Stop,Café,Fried Chicken Joint,Boutique,Pub,Waterfront,Electronics Store,Department Store,Dessert Shop,Discount Store


In [75]:
Newham_merged.loc[Newham_merged['Cluster Labels'] == 3, Newham_merged.columns[[1] + list(range(5, Newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Manor Park,Gym / Fitness Center,Grocery Store,Train Station,Restaurant,English Restaurant,Dance Studio,Department Store,Dessert Shop,Discount Store,Doner Restaurant


In [76]:
Newham_merged.loc[Newham_merged['Cluster Labels'] == 4, Newham_merged.columns[[1] + list(range(5, Newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,West Ham,Park,Asian Restaurant,Bus Station,Breakfast Spot,Bulgarian Restaurant,English Restaurant,Department Store,Dessert Shop,Discount Store,Doner Restaurant


# Conclusion: 

After examining the above 5 clusters, we can recommend our stakeholders that Beckton,Custom House, Maryland, Eastham and Manor Park are the best neighborhoods in Newham borough, to open their asian restuarant. This is because in these areas, the most common venue visited by the public is the restuarants and as these areas are highly populated with asians, opening an asian restuarant would definitley be a good idea.